# Numbers

Scrpt to generate basic numbers such as the number of mice and the number of cells per mouse.

In [45]:
import sys
import pandas as pd
import numpy as np
sys.path.append("/Users/federicoclaudi/Documents/Github/LocomotionControl")

from data.dbase.db_tables import (
    Session,
    Mouse,
    ValidatedSession,
    Tracking,
    SessionCondition,
    LocomotionBouts,
    Surgery, 
    Probe, 
    Recording, 
    Unit,
)

In [2]:
# get the number of sessions that are implanted vs baseline
sessions = pd.DataFrame(ValidatedSession * SessionCondition)

baseline = sessions[sessions["condition"] == "naive"]
implanted = sessions[sessions["condition"] != "naive"]

print(f"Of a total of {len(sessions)} sessions, {len(baseline)} are baseline and {len(implanted)} are implanted")

# for each group of session, get the number of mice
baseline_mice = len(baseline["mouse_id"].unique())
implanted_mice = len(implanted["mouse_id"].unique())
print(f"Of a total of {len(sessions['mouse_id'].unique())} mice, {baseline_mice} are baseline and {implanted_mice} are implanted")

# get the average and std of the number of sessions per mouse
baseline_mu, baseline_std = np.mean(baseline.groupby("mouse_id").count()["name"].values), np.std(baseline.groupby("mouse_id").count()["name"].values)
implanted_mu, implanted_std = np.mean(implanted.groupby("mouse_id").count()["name"].values), np.std(implanted.groupby("mouse_id").count()["name"].values)

print(f"Baseline mice have {baseline_mu:.2f} +/- {baseline_std:.2f} sessions, while implanted mice have {implanted_mu:.2f} +/- {implanted_std:.2f} sessions")


Of a total of 234 sessions, 154 are baseline and 80 are implanted
Of a total of 20 mice, 20 are baseline and 7 are implanted
Baseline mice have 7.70 +/- 6.82 sessions, while implanted mice have 11.43 +/- 5.60 sessions


## Locomotion bouts

In [80]:
# Get the number of trials (locomotion bouts) per session
print("Getting number of locomotion bouts for BASELINE sessions")
for direction in ["inbound", "outbound"]:
    bouts = pd.DataFrame(LocomotionBouts * Session * SessionCondition  & "complete='true'" & f"direction='{direction}'" & "duration<12"  & "gcoord_delta > 0.925")
    bouts = bouts[bouts["condition"] == "naive"]
    
    print(f"Tot number of {direction} bouts: {len(bouts)} - average duration {np.mean(bouts['duration'].values):.2f} +/- {np.std(bouts['duration'].values):.2f} s")

    n_per_mouse = bouts.groupby("mouse_id").count()["name"]
    print(f"    Average number of {direction} bouts per mouse: {np.mean(n_per_mouse.values):.2f} +/- {np.std(n_per_mouse.values):.2f} ")

    n_per_session = bouts.groupby("name").count()["duration"]
    print(f"    Average number of {direction} bouts per session: {np.mean(n_per_session.values):.2f} +/- {np.std(n_per_session.values):.2f} ")

# TODO add average duration

Getting number of locomotion bouts for baseline sessions

Tot number of inbound bouts: 1963 - average duration 6.42 +/- 1.73 s

Average number of inbound bouts per mouse: 98.15 +/- 91.36

Average number of inbound bouts per session: 19.83 +/- 16.85

Tot number of outbound bouts: 3195 - average duration 6.47 +/- 1.65 s

Average number of outbound bouts per mouse: 159.75 +/- 126.89

Average number of outbound bouts per session: 31.32 +/- 20.78

In [66]:
# Get the number of trials (locomotion bouts) per session
print("Getting number of locomotion bouts for RECORDING sessions")
#TODO make sure we have bouts for all recording mice
#TODO break down number of bouts per session per group of mice

summary = dict(
    target=[], outbound_tot=[], outbound_per_mouse=[], outbound_per_session=[], inbound_tot=[], inbound_per_mouse=[], inbound_per_session=[]
)
for target in ("MOs", "CUN/GRN"):
    for direction in ["inbound", "outbound"]:

        mice = (Surgery & f"target='{target}'").fetch("mouse_id")


        bouts = pd.DataFrame(LocomotionBouts * Surgery * SessionCondition  & "complete='true'" & f"direction='{direction}'" & "duration<12" & f"target='{target}'"  & "gcoord_delta > 0.925")
        # bouts = bouts[bouts["condition"] != "naive"]

        bouts_mice = bouts["mouse_id"].unique()
        print(f"Mice with implant probe: {mice} - we have bouts for: {bouts_mice}")
        break
    break
        
        # if direction == "inbound":
        #     summary["target"].append(target)

        # print(target, bouts.groupby("mouse_id").count()["name"].values)
        # TODO add duration

        # summary[f"{direction}_tot"].append(len(bouts))
        # summary[f"{direction}_per_mouse"].append(np.mean(bouts.groupby("mouse_id").count()["name"].values))
        # summary[f"{direction}_per_session"].append(np.mean(bouts.groupby("name").count()["duration"].values))



        
        # print(f"Tot number of {target} - {direction} bouts: {len(bouts)}")

        # n_per_mouse = bouts.groupby("mouse_id").count()["name"]
        # print(f"    Average number of {target} - {direction} bouts per mouse: {np.mean(n_per_mouse.values):.2f} +/- {np.std(n_per_mouse.values):.2f}")

        # n_per_session = bouts.groupby("name").count()["duration"]
        # print(f"    Average number of {target} - {direction} bouts per session: {np.mean(n_per_session.values):.2f} +/- {np.std(n_per_session.values):.2f}")


Getting number of locomotion bouts for RECORDING sessions

Mice with implant probe: ['BAA0000012' 'BAA1101192'] - we have bouts for: ['BAA0000012' 'BAA1101192']

In [53]:
pd.DataFrame(summary)

,target,outbound_tot,outbound_per_mouse,outbound_per_session,inbound_tot,inbound_per_mouse,inbound_per_session
0,MOs,444,444.000000,29.600000,334,334.0,22.266667
1,CUN/GRN,322,53.666667,9.757576,189,31.5,6.096774


### units

NameError: name 'unique' is not defined

In [4]:
mos_regions = ["MOs1", "MOs2/3", "MOs5", "MOs6a"]
mrn_regions = ["PPN", "CUN"]
rsp_regions = ['RSPagl2/3', 'RSPagl5', 'RSPagl6a', 'RSPagl6b', 'RSPd1', 'RSPd2/3', 'RSPd5', 'RSPd6a']
vis_regions = ['VISp1', 'VISp2/3', 'VISp4', 'VISp5', 'VISp6a']
orb_regions = ['ORBl5', 'ORBl6a', 'ORBvl2/3', 'ORBvl5', 'ORBvl6a']

brain_regions = dict(
    mos=mos_regions,
    mrn=mrn_regions,
    rsp=rsp_regions,
    vis=vis_regions,
    orb=orb_regions,
)

In [5]:
# Get the number of recordings per implanted mice.

mos_mice = (Mouse * Surgery & "target='MOs'")
mrn_mice = (Mouse * Surgery & "target='CUN/GRN'")

print(f"A total of {len(mos_mice)} mice were implanted in MOs, {len(mrn_mice)} were implanted in MRN")

# Get the number of recordings per session.
mos_recs = pd.DataFrame((Recording * Surgery & "target='MOs'").fetch())
mrn_recs = pd.DataFrame((Recording * Surgery & "target='CUN/GRN'").fetch())

print(f"A total of {len(mos_recs)} recordings were made in MOs, {len(mrn_recs)} recordings were made in MRN")

# get the average and std of the number of recordings per session
mos_mu, mos_std = mos_recs.groupby("mouse_id").count().name.mean(), mos_recs.groupby("mouse_id").count().name.std()
mrn_mu, mrn_std = mrn_recs.groupby("mouse_id").count().name.mean(), mrn_recs.groupby("mouse_id").count().name.std()

print(f"{mos_mu:.2f} +/- {mos_std:.2f} recordings per mouse for MOs, {mrn_mu:.2f} +/- {mrn_std:.2f} recordings per mouse for MRN")


# Get the number of units per brain region
units = pd.DataFrame((Unit * Probe.RecordingSite ).fetch())
print("\n\n")
for brain_region, subregions in brain_regions.items():
    units_in_region = units[units['brain_region'].isin(subregions)]
    print(f"Number of units in {brain_region}: {len(units_in_region)}")

    in_region_by_mouse = units_in_region.groupby('mouse_id').count().name.values
    print(f"  Number of units per mouse: {np.mean(in_region_by_mouse):.2f} +/- {np.std(in_region_by_mouse):.2f} ")

    for subregion in subregions:
        n = len(units[units['brain_region'] == subregion])
        if n == 0:
            continue
        print(f"\t{subregion}: {n}")

A total of 2 mice were implanted in MOs, 6 were implanted in MRN
A total of 12 recordings were made in MOs, 20 recordings were made in MRN
12.00 +/- nan recordings per mouse for MOs, 4.00 +/- 2.45 recordings per mouse for MRN



Number of units in mos: 475
	MOs1: 97
	MOs2/3: 99
	MOs5: 169
	MOs6a: 110
Number of units in mrn: 185
	PPN: 75
	CUN: 110
Number of units in rsp: 16
	RSPagl2/3: 7
	RSPagl6b: 4
	RSPd1: 3
	RSPd2/3: 2
Number of units in vis: 23
	VISp1: 18
	VISp6a: 5
Number of units in orb: 73
	ORBl5: 4
	ORBl6a: 36
	ORBvl5: 2
	ORBvl6a: 31


In [86]:
print("Number of recordings per mouse")
pd.DataFrame(Recording().fetch()).groupby("mouse_id").count()["name"]

,name,concatenated,spike_sorting_params_file_path,spike_sorting_spikes_file_path,spike_sorting_clusters_file_path,recording_probe_configuration,reference
mouse_id,,,,,,,
AAA1110750,5,5,5,5,5,5,5
BAA1101192,12,12,12,12,12,12,12
BAA110516,2,2,2,2,2,2,2
BAA110517,5,5,5,5,5,5,5
BAA1110279,1,1,1,1,1,1,1
BAA1110281,7,7,7,7,7,7,7
